In [10]:
# Import necessary libraries and modules
import pickle
import numpy
from music21 import *
import random
import string
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.layers import BatchNormalization as BatchNorm
from google.colab import drive

# Define the MusicGenerator class
class MusicGenerator:
    def __init__(self):
        # Initialize the MusicGenerator class
        self.model = None
        self.pitch = None
        self.amount_of_pitch = None
        self.niput = None
        drive.mount('/content/drive')  # Mount Google Drive to access files

    def load_data(self, data_filepath):
        # Load musical data from a pickle file
        with open(data_filepath, 'rb') as file:
            data = pickle.load(file)

        # Extract unique pitches and prepare input sequences
        self.pitch = sorted(set(item for item in data))
        self.amount_of_pitch = len(set(data))
        self.niput, _ = self.sequence(data)

    def sequence(self, data):
        # Create input-output sequences for training the model
        squelength = 100
        daoin = dict((note, number) for number, note in enumerate(self.pitch))
        niput = []
        output = []
        for i in range(0, len(data) - squelength, 1):
            squein = data[i:i + squelength]
            squeout = data[i + squelength]
            niput.append([daoin[char] for char in squein])
            output.append([daoin[squeout]])
        ilength = len(niput)
        nniput = numpy.reshape(niput, (ilength, squelength, 1))
        nniput = nniput / float(self.amount_of_pitch)
        output = numpy.array(output)
        return nniput, output

    def create_model(self):
        # Create and load the LSTM model with pre-trained weights
        model = Sequential()
        model.add(LSTM(512, input_shape=(self.niput.shape[1], self.niput.shape[2]), recurrent_dropout=0.3, return_sequences=True))
        model.add(LSTM(512, recurrent_dropout=0.3, return_sequences=True,))
        model.add(LSTM(512))
        model.add(BatchNorm())
        model.add(Dropout(0.3))
        model.add(Dense(256))
        model.add(Activation('relu'))
        model.add(BatchNorm())
        model.add(Dropout(0.3))
        model.add(Dense(self.amount_of_pitch))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
        model.load_weights('/content/drive/MyDrive/project/weight/weights.hdf5')
        model.summary()
        self.model = model

    def create_random_notes(self, length=50, data=None):
    # Generate a random sequence of notes
        if data is None:
             data = self.pitch

        random_indices = random.sample(range(len(data)), length)
        itnot = dict((number, note) for number, note in enumerate(data))
        preoutput = [itnot[index] for index in random_indices]
        return preoutput

    def create_fixed_sequence_notes(self):
        # Generate a fixed sequence of notes using the trained model
        start = numpy.random.randint(0, len(self.niput) - 1)
        itnot = dict((number, note) for number, note in enumerate(self.pitch))
        shape = self.niput[start].tolist()
        preoutput = []
        for notei in range(50):
            preinput = numpy.reshape(shape, (1, len(shape), 1))
            prediction_input = preinput / float(self.amount_of_pitch)
            pre = self.model.predict(prediction_input, verbose=0)
            index = numpy.argmin(pre)
            result = itnot[index]
            preoutput.append(result)
            shape = numpy.append(shape, index)
            shape = shape[1:]
        return preoutput

    def create_song(self, preoutput):
        # Convert the generated notes into a Music21 stream and save as a MIDI file
        offset = 0
        outputn = []
        for shape in preoutput:
            if ('.' in shape) or shape.isdigit():
                nic = shape.split('.')
                data = []
                for currentdata in nic:
                    latestdata = note.Note(int(currentdata))
                    latestdata.storedInstrument = instrument.Piano()
                    data.append(latestdata)
                latestchord = chord.Chord(data)
                latestchord.offset = offset
                outputn.append(latestchord)
            else:
                latestdata = note.Note(shape)
                latestdata.offset = offset
                latestdata.storedInstrument = instrument.Piano()
                outputn.append(latestdata)
            offset += 0.5

        random_suffix = ''.join(random.choices(string.ascii_letters + string.digits, k=8))
        output_filepath = f'/content/drive/MyDrive/project/output/output_{random_suffix}.mid'
        song = stream.Stream(outputn)
        song.write('midi', fp=output_filepath)
        print(f"MIDI file generated: {output_filepath}")

    def random_generation(self, length=50, data_filepath=None):
    # If a data_filepath is provided, load the data from that path
        if data_filepath:
          self.load_data(data_filepath)

    # Generate random notes from the loaded data and create a song
        preoutput = self.create_random_notes(length)
        self.create_song(preoutput)
        print("Generated (Random):", preoutput)
        print("Music generated successfully!")


    def fixed_sequence_generation(self):
        # Generate a fixed sequence of notes using the trained model and create a song
        fixed_sequence = self.create_fixed_sequence_notes()
        self.create_song(fixed_sequence)
        print("Generated (Fixed Sequence):", fixed_sequence)
        print("Music generated successfully!")

    def predict_next_part(self):
        # Predict the next part of the song and create a song
        if self.model is not None:
            default_predict_length = 100
            user_input_predict_length = input(f"Enter the length for prediction (default is {default_predict_length}): ")
            predict_length = int(user_input_predict_length) if user_input_predict_length.isdigit() else default_predict_length

            preoutput = self.create_random_notes(length=predict_length)
            self.create_song(preoutput)
            print("Predicted Next Part (Random):", preoutput)
            print("Music generated successfully!")
        else:
            print("Model not loaded. Please load the model first.")

# Main function
def main():
    music_generator = MusicGenerator()
    try:
        # Load data, create model, and perform music generation based on user input
        data_filepath = '/content/drive/MyDrive/project/model/training_data'
        music_generator.load_data(data_filepath)
        music_generator.create_model()
        action = int(input("Choose an action:\n1. Random Generation\n2. Fixed Sequence Generation\n3. Predict Next Part (Random)\nEnter the number: "))

        if action == 1:
            music_generator.random_generation()
        elif action == 2:
            music_generator.fixed_sequence_generation()
        elif action == 3:
            music_generator.predict_next_part()
        else:
            print("Invalid action. Please choose a valid option.")

    except Exception as e:
        print(f"An error occurred: {e}")

# Execute the main function if the script is run directly
if __name__ == '__main__':
    main()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_21 (LSTM)              (None, 100, 512)          1052672   
                                                                 
 lstm_22 (LSTM)              (None, 100, 512)          2099200   
                                                                 
 lstm_23 (LSTM)              (None, 512)               2099200   
                                                                 
 batch_normalization_14 (Ba  (None, 512)               2048      
 tchNormalization)                                               
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                         